In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import math

In [2]:
MON = pd.read_excel('OViN2017_Databestand.xlsx', sheet_name = 'Databestand')

In [3]:
MON1 = MON

In [4]:
number_persons = len(list(MON1['OPID'].unique()))
number_persons

38127

In [5]:
# MON = MON['OPID', 'Jaardag', 'VertUur', 'AankUur', 'VertMin', 'AankMin', 'BVertUur', 'BVertMin', 'BAankUur', 'BAankMin', 'RVertUur', 'RVertMin', 'RAankUur', 'RAankMin', 'Rvm', 'Doel']    Hier nog alle nuttige kolommen selecteren
MON1 = MON1[MON1['Rvm'] == 6]
MON1 = MON1.reset_index()

In [6]:
MON1['RVertUur'] = MON1['RVertUur'].fillna(MON1['BVertUur'])
MON1['RVertMin'] = MON1['RVertMin'].fillna(MON1['BVertMin'])
MON1['RAankUur'] = MON1['RAankUur'].fillna(MON1['BAankUur'])
MON1['RAankMin'] = MON1['RAankMin'].fillna(MON1['BAankMin'])

MON1['RVertUur'] = MON1['RVertUur'].fillna(MON1['VertUur'])
MON1['RVertMin'] = MON1['RVertMin'].fillna(MON1['VertMin'])
MON1['RAankUur'] = MON1['RAankUur'].fillna(MON1['AankUur'])
MON1['RAankMin'] = MON1['RAankMin'].fillna(MON1['AankMin'])

MON1 = MON1.dropna(subset=['RVertUur'])
MON1 = MON1.dropna(subset=['RAankUur'])
MON1 = MON1.reset_index()
MON1 = MON1.drop(['level_0', 'index'], axis=1)

In [28]:
MON1.loc[MON1['RAankUur'] > 23, 'RAankUur'] = (MON1['RAankUur'] - 24)

In [29]:
daylist = list(MON1['Jaardag'].unique())
daylist.sort()

timelist = []
for a in range(0, 24):
    for b in range(0, 59, 15):
        t = dt.time(hour = a, minute = b)
        timelist.append(t)

personlist = list(MON1['OPID'].unique())

In [30]:
ind = list(range(96 * len(daylist)))
col = ['Home', 'Work', 'Work Visit', 'Work Transport', 'Bringing', 'Education', 'Shop', 'Visit', 'Wandering', 'Hobby', 'Leisure', 'Personal Care', 'Other', 'Driving', 'Number Respondents Driving', 'Number Respondents']
df9 = pd.DataFrame(index = ind, columns = col)

In [32]:
Index_Teller = 0
for d in daylist:
    print(d)
    df3 = MON[MON['Jaardag'] == d]
    number_respondents = len(list(df3['OPID'].unique()))
    df1 = MON1[MON1['Jaardag'] == d]
    number_respondents_driving = len(list(df1['OPID'].unique()))
    personlist = list(df1['OPID'].unique())
    for t in timelist:
        number_home = 0
        number_working = 0
        number_work_visiting = 0
        number_work_transport = 0
        number_bringing = 0
        number_education = 0
        number_shopping = 0
        number_visiting = 0
        number_wandering = 0
        number_hobbying = 0
        number_leisure = 0
        number_personal_care = 0
        number_other = 0
        number_driving = 0
        for p in personlist:
            df2 = df1[df1['OPID'] == p]
            df2 = df2.reset_index()
            if math.isnan(df2.loc[0, 'RVertUur']):
                number_home += 1
            elif t < dt.time(hour = int(df2.loc[0, 'RVertUur']), minute = int(df2.loc[0, 'RVertMin'])):
                number_home += 1
            elif t >= dt.time(hour = int(df2.loc[(df2.shape[0] - 1), 'RAankUur']), minute = int(df2.loc[(df2.shape[0] - 1), 'RAankMin'])):
                number_home += 1                                                      # Als ze altijd thuis slapen?
            elif t < dt.time(hour = int(df2.loc[0, 'RAankUur']), minute = int(df2.loc[0, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[1, 'RVertUur']), minute = int(df2.loc[1, 'RVertMin'])):
                if df2.loc[0, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[0, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[0, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[0, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[0, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[0, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[0, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[0, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[0, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[0, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[0, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[0, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[1, 'RAankUur']), minute = int(df2.loc[1, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[2, 'RVertUur']), minute = int(df2.loc[2, 'RVertMin'])):
                if df2.loc[1, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[1, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[1, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[1, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[1, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[1, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[1, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[1, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[1, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[1, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[1, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[1, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
               
            elif t < dt.time(hour = int(df2.loc[2, 'RAankUur']), minute = int(df2.loc[2, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[3, 'RVertUur']), minute = int(df2.loc[3, 'RVertMin'])):
                if df2.loc[2, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[2, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[2, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[2, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[2, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[2, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[2, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[2, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[2, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[2, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[2, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[2, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[3, 'RAankUur']), minute = int(df2.loc[3, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[4, 'RVertUur']), minute = int(df2.loc[4, 'RVertMin'])):
                if df2.loc[3, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[3, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[3, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[3, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[3, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[3, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[3, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[3, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[3, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[3, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[3, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[3, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[4, 'RAankUur']), minute = int(df2.loc[4, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[5, 'RVertUur']), minute = int(df2.loc[5, 'RVertMin'])):
                if df2.loc[4, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[4, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[4, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[4, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[4, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[4, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[4, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[4, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[4, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[4, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[4, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[4, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[5, 'RAankUur']), minute = int(df2.loc[5, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[6, 'RVertUur']), minute = int(df2.loc[6, 'RVertMin'])):
                if df2.loc[5, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[5, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[5, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[5, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[5, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[5, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[5, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[5, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[5, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[5, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[5, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[5, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[6, 'RAankUur']), minute = int(df2.loc[6, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[7, 'RVertUur']), minute = int(df2.loc[7, 'RVertMin'])):
                if df2.loc[6, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[6, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[6, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[6, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[6, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[6, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[6, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[6, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[6, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[6, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[6, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[6, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[7, 'RAankUur']), minute = int(df2.loc[7, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[8, 'RVertUur']), minute = int(df2.loc[8, 'RVertMin'])):
                if df2.loc[7, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[7, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[7, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[7, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[7, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[7, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[7, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[7, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[7, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[7, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[7, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[7, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[8, 'RAankUur']), minute = int(df2.loc[8, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[9, 'RVertUur']), minute = int(df2.loc[9, 'RVertMin'])):
                if df2.loc[8, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[8, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[8, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[8, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[8, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[8, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[8, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[8, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[8, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[8, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[8, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[8, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[9, 'RAankUur']), minute = int(df2.loc[9, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[10, 'RVertUur']), minute = int(df2.loc[10, 'RVertMin'])):
                if df2.loc[9, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[9, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[9, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[9, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[9, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[9, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[9, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[9, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[9, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[9, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[9, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[9, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[10, 'RAankUur']), minute = int(df2.loc[10, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[11, 'RVertUur']), minute = int(df2.loc[11, 'RVertMin'])):
                if df2.loc[10, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[10, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[10, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[10, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[10, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[10, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[10, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[10, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[10, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[10, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[10, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[10, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[11, 'RAankUur']), minute = int(df2.loc[11, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[12, 'RVertUur']), minute = int(df2.loc[12, 'RVertMin'])):
                if df2.loc[11, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[11, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[11, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[11, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[11, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[11, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[11, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[11, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[11, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[11, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[11, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[11, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[12, 'RAankUur']), minute = int(df2.loc[12, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[13, 'RVertUur']), minute = int(df2.loc[13, 'RVertMin'])):
                if df2.loc[12, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[12, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[12, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[12, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[12, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[12, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[12, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[12, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[12, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[12, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[12, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[12, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[13, 'RAankUur']), minute = int(df2.loc[13, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[14, 'RVertUur']), minute = int(df2.loc[14, 'RVertMin'])):
                if df2.loc[13, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[13, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[13, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[13, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[13, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[13, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[13, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[13, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[13, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[13, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[13, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[13, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[14, 'RAankUur']), minute = int(df2.loc[14, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[15, 'RVertUur']), minute = int(df2.loc[15, 'RVertMin'])):
                if df2.loc[14, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[14, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[14, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[14, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[14, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[14, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[14, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[14, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[14, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[14, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[14, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[14, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[15, 'RAankUur']), minute = int(df2.loc[15, 'RAankMin'])):
                number_driving += 1
            else: 
                number_home += 1
                print('Something is going wrong')
                
        df9.loc[Index_Teller, 'Home'] = number_home
        df9.loc[Index_Teller, 'Work'] = number_working
        df9.loc[Index_Teller, 'Work Visit'] = number_work_visiting
        df9.loc[Index_Teller, 'Work Transport'] = number_work_transport
        df9.loc[Index_Teller, 'Bringing'] = number_bringing
        df9.loc[Index_Teller, 'Education'] = number_education
        df9.loc[Index_Teller, 'Shop'] = number_shopping
        df9.loc[Index_Teller, 'Visit'] = number_visiting
        df9.loc[Index_Teller, 'Wandering'] = number_wandering
        df9.loc[Index_Teller, 'Hobby'] = number_hobbying
        df9.loc[Index_Teller, 'Leisure'] = number_leisure
        df9.loc[Index_Teller, 'Personal Care'] = number_personal_care
        df9.loc[Index_Teller, 'Other'] = number_other
        df9.loc[Index_Teller, 'Driving'] = number_driving
        df9.loc[Index_Teller, 'Number Respondents Driving'] = number_respondents_driving
        df9.loc[Index_Teller, 'Number Respondents'] = number_respondents
        
        Index_Teller += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# Save results

In [33]:
df9.to_csv('Behaviour all carsjaardag OUD.csv')

In [34]:
ind9 = (pd.DataFrame(columns=['NULL'],index=pd.date_range('2017-01-01 00:00', '2017-12-31 23:45',freq='15T')).index.strftime('%Y-%m-%d %H:%M:%S').tolist())
df9['index'] = ind9
df9 = df9.set_index('index')

In [35]:
df9.to_csv('Behaviour all cars jaardag nieuw.csv')

# Percentages

In [36]:
# df = pd.read_csv('Behaviour all cars jaardag.csv')
# df = df.set_index('index')
# df['Number Respondents'] = df.sum(axis = 1, skipna = True) 

df_percentages = df9

for i in list(df_percentages.columns[0:14]):
    df_percentages[i] = df_percentages[i] / df_percentages['Number Respondents Driving']

In [37]:
df_percentages.to_csv('Percentages nieuw.csv')

In [9]:
df9 = pd.read_csv('Behaviour all cars jaardag.csv')
df9 = df9.set_index('index')

In [11]:
df9[]

,Home,Work,Work Visit,Work Transport,Bringing,Education,Shop,Visit,Wandering,Hobby,Leisure,Personal Care,Other,Driving,Number Respondents Driving,Number Respondents
index,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,24,0,0,0,0,0,0,0,0,0,0,0,0,0,24,111
2017-01-01 00:15:00,24,0,0,0,0,0,0,0,0,0,0,0,0,0,24,111
2017-01-01 00:30:00,24,0,0,0,0,0,0,0,0,0,0,0,0,0,24,111
2017-01-01 00:45:00,24,0,0,0,0,0,0,0,0,0,0,0,0,0,24,111
2017-01-01 01:00:00,24,0,0,0,0,0,0,0,0,0,0,0,0,0,24,111


In [5]:
df_percentages = pd.read_csv('Percentages nieuw.csv')
df_percentages = df_percentages.set_index('index')

In [8]:
df_percentages[225:240]

,Home,Work,Work Visit,Work Transport,Bringing,Education,Shop,Visit,Wandering,Hobby,Leisure,Personal Care,Other,Driving,Number Respondents Driving,Number Respondents
index,,,,,,,,,,,,,,,,
2017-01-03 09:30:00,0.482759,0.413793,0.034483,0.0,0.000000,0.0,0.034483,0.0,0.0,0.0,0.0,0.034483,0.0,0.000000,29,95
2017-01-03 09:45:00,0.413793,0.413793,0.034483,0.0,0.000000,0.0,0.034483,0.0,0.0,0.0,0.0,0.034483,0.0,0.068966,29,95
2017-01-03 10:00:00,0.344828,0.413793,0.000000,0.0,0.034483,0.0,0.034483,0.0,0.0,0.0,0.0,0.034483,0.0,0.137931,29,95
2017-01-03 10:15:00,0.344828,0.413793,0.000000,0.0,0.068966,0.0,0.034483,0.0,0.0,0.0,0.0,0.034483,0.0,0.103448,29,95
2017-01-03 10:30:00,0.344828,0.413793,0.000000,0.0,0.103448,0.0,0.034483,0.0,0.0,0.0,0.0,0.034483,0.0,0.068966,29,95
2017-01-03 10:45:00,0.344828,0.413793,0.000000,0.0,0.103448,0.0,0.034483,0.0,0.0,0.0,0.0,0.034483,0.0,0.068966,29,95
2017-01-03 11:00:00,0.310345,0.413793,0.000000,0.0,0.103448,0.0,0.034483,0.0,0.0,0.0,0.0,0.034483,0.0,0.103448,29,95
2017-01-03 11:15:00,0.310345,0.413793,0.000000,0.0,0.103448,0.0,0.034483,0.0,0.0,0.0,0.0,0.034483,0.0,0.103448,29,95
2017-01-03 11:30:00,0.310345,0.413793,0.000000,0.0,0.103448,0.0,0.034483,0.0,0.0,0.0,0.0,0.034483,0.0,0.103448,29,95


# Calculate max number of trips

In [ ]:
List = list(MON['OPID'])

In [ ]:
def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
            print('update')
  
    return num

In [ ]:
most_frequent(List)

# Save codes

In [ ]:
# l = (pd.DataFrame(columns=['NULL'],
#                   index=pd.date_range('00:00', '23:45',
#                                       freq='15T'))
#    #    .between_time('07:00','21:00')
#        .index.strftime('%H:%M')
#        .tolist()
# )


# l = (pd.DataFrame(columns=['NULL'],
#                   index=pd.date_range('2016-09-02 17:30:00', '2016-09-04 21:00:00',
#                                       freq='15T'))
#    #    .between_time('07:00','21:00')
#        .index.strftime('%Y-%m-%d %H:%M:%S')
#        .tolist()
# )
# l

In [ ]:
# number_respondents_list = []
# for d in daylist:
#     df1 = MON1[MON1['Jaardag'] == d]
#     number_respondents = len(list(df1['OPID'].unique()))
#     L = [number_respondents] * 96
#     number_respondents_list = number_respondents_list + L